### Sequence Clustering

This notebook aims to experiment with clustering different character sequences.

In [4]:
import numpy as np
import random

To achieve this, we will compare words (finite sequences of letters) using the Edit Distance, a metric that quantifies the minimum number of operations (insertions, deletions, or substitutions) required to transform one sequence into another.

While the recursive algorithm for calculating the Edit Distance works, we observed that it often performs redundant and unnecessary calculations. To address this, we implement memoization, a technique that stores intermediate results to avoid redundant computations. This optimization reduces the complexity to O(n × m), where n=∣u∣ and m=∣v∣ represent the lengths of the sequences being compared. 

In [5]:
def edit_distance(seq1, seq2):
    seq1 = seq1.split()
    seq2 = seq2.split()
    
    memo = {}

    def dist(i, j):
        if (i, j) in memo:
            return memo[(i, j)]

        if i == len(seq1):
            return len(seq2) - j
        if j == len(seq2):
            return len(seq1) - i

        if seq1[i] == seq2[j]:
            memo[(i, j)] = dist(i + 1, j + 1)
        else:
            insert = dist(i, j + 1)
            delete = dist(i + 1, j)
            substitute = dist(i + 1, j + 1)
            memo[(i, j)] = 1 + min(insert, delete, substitute)

        return memo[(i, j)]

    return dist(0, 0)


In [6]:
edit_distance('A C JM JO JU KD JW KB KE KC JZ JX JV JP KF KH KQ LF KJ LB LJ KK LM LD KI LE LK KL LC LG KR KO KN KP KM KG JN D B',
              'A C JM JO JU KD JW BK KE KC JZ JX JV JP KF KH KQ FO KJ LB LJ KK LM D KI LE LK KL LC LG R KO KN KP KM KG JN D B')

4

#### Clustering with the k-Medoids Technique

We then explore the k-Medoids clustering method, which is conceptually similar to k-Means but differs in how cluster representatives are chosen. Instead of using centroids (barycenters), k-Medoids selects actual data points as cluster representatives, making it more robust to noise and outliers. We also compute silhouette scores to evaluate the quality of the clustering.

In [7]:
def assign_clusters(sequences, medoids, distance_fn):
    clusters = {medoid: [] for medoid in medoids}
    for seq in sequences:
        closest_medoid = min(medoids, key=lambda m: distance_fn(seq, m))
        clusters[closest_medoid].append(seq)
    return clusters

def calculate_new_medoid(cluster, distance_fn):
    return min(cluster, key=lambda candidate: sum(distance_fn(candidate, seq) for seq in cluster))

def k_medoids(sequences, k, distance_fn, max_iter=200):
    medoids = random.sample(sequences, k)

    for _ in range(max_iter):
        clusters = assign_clusters(sequences, medoids, distance_fn)

        new_medoids = [calculate_new_medoid(cluster, distance_fn) for cluster in clusters.values()]

        if set(new_medoids) == set(medoids):
            break

        medoids = new_medoids

    return clusters, medoids


In [8]:
def load_sequences(file_path, num_sequences=100):
    with open(file_path, 'r') as file:
        all_sequences = [line.strip() for line in file.readlines()]
        return random.sample(all_sequences, min(len(all_sequences), num_sequences))

In [9]:
sequences = load_sequences("log.txt", num_sequences=100)

k = 3
clusters, medoids = k_medoids(sequences, k, edit_distance)

print("Results of k-medoid clustering :\n")

for i, (medoid, cluster) in enumerate(clusters.items(), 1):
    print(f"Cluster {i} :")
    print(f"  Medoid: {medoid}")
    print(f"  Number of sequences: {len(cluster)}")
    print(f"  Sequences :")
    
    for seq in cluster:
        print(f"    {seq}")
    
    print("\n" + "-"*50 + "\n")

Results of k-medoid clustering :

Cluster 1 :
  Medoid: A C E G T V X W U Y AA Z H AC AO AE AQ AG AR AI AP AH AS AF AV AT AD F D B
  Number of sequences: 65
  Sequences :
    A C E G I K M L J H AC AE AO AG AR AQ AI AP AH AS AF AV AT AD F D B
    A C EC HL HV HX HZ HY HW HM IU IW IX IY JB IZ IV ED D B
    A C EC FL FS FN FX FU FV FW FZ FQ FP FR FY FT FO FM GC GJ GE GK GG GI GH GF GD GL GN GO GM GW GY HA HB GZ GX ED D B
    A C EC HL HV HX IB HY HW HM IU JC JG JE JK JI JJ JL JF JH JD IV ED D B
    A C EC EE EI EM EJ EF EN ER EP EQ ES EO ET EV EX EY EW EU EZ FG FI FJ FK FH FA ED D B
    A C E G T V W X U Y AA Z H AC AE AO AG AQ AJ AR AP AH AS AF AW AT AD F D B
    A C E AX CA CR AZ DI CC BB DP CD BH CE DS DR DT BL CF DQ BI DJ CB BC CS BA AY F D B
    A C E AX CR CA AZ DW BM CG DY CI BO EA DZ CK BR CL EB DX BP CJ BN CH CS BA CB AY F D B
    A C E AX CR AZ DW CA BB EA DY CG EB CI DZ BH BK CK CL DX BI CJ CS CH BC BA CB AY F D B
    A C EC EE EG EH EF EN EP ER EQ ES EO ET EX EV EY EW EU EZ F

In [10]:
def calculate_silhouette_scores(clusters, distance_fn):
    silhouette_scores = {}
    all_sequences = [seq for cluster in clusters.values() for seq in cluster]

    for medoid, cluster in clusters.items():
        for seq in cluster:
            intra_distances = [distance_fn(seq, other_seq) for other_seq in cluster if seq != other_seq]
            a_i = np.mean(intra_distances) if intra_distances else 0
            
            inter_distances = []
            for other_medoid, other_cluster in clusters.items():
                if other_medoid != medoid:
                    inter_distances.append(np.mean([distance_fn(seq, other_seq) for other_seq in other_cluster]))
            b_i = min(inter_distances) if inter_distances else 0
            
            if a_i < b_i:
                s_i = (b_i - a_i) / b_i
            elif a_i > b_i:
                s_i = (b_i - a_i) / a_i
            else:
                s_i = 0 
            
            silhouette_scores[seq] = s_i

    avg_silhouette_scores = {
        medoid: np.mean([silhouette_scores[seq] for seq in cluster])
        for medoid, cluster in clusters.items()
    }

    overall_silhouette_score = np.mean(list(silhouette_scores.values()))

    return silhouette_scores, avg_silhouette_scores, overall_silhouette_score



In [11]:
silhouette_scores, avg_silhouette_scores, overall_silhouette_score = calculate_silhouette_scores(clusters, edit_distance)

print("Silhouette scores for each point :")
for seq, score in silhouette_scores.items():
    print(f"  Sequence: {seq} -> Silhouette score: {score:.3f}")

print("\nAverage silhouette score per cluster :")
for medoid, score in avg_silhouette_scores.items():
    print(f"  Medoid: {medoid} -> Average score: {score:.3f}")

print(f"\nOverall silhouette score: {overall_silhouette_score:.3f}")

Silhouette scores for each point :
  Sequence: A C E G I K M L J H AC AE AO AG AR AQ AI AP AH AS AF AV AT AD F D B -> Silhouette score: -0.510
  Sequence: A C EC HL HV HX HZ HY HW HM IU IW IX IY JB IZ IV ED D B -> Silhouette score: 0.006
  Sequence: A C EC FL FS FN FX FU FV FW FZ FQ FP FR FY FT FO FM GC GJ GE GK GG GI GH GF GD GL GN GO GM GW GY HA HB GZ GX ED D B -> Silhouette score: 0.099
  Sequence: A C EC HL HV HX IB HY HW HM IU JC JG JE JK JI JJ JL JF JH JD IV ED D B -> Silhouette score: 0.033
  Sequence: A C EC EE EI EM EJ EF EN ER EP EQ ES EO ET EV EX EY EW EU EZ FG FI FJ FK FH FA ED D B -> Silhouette score: 0.061
  Sequence: A C E G T V W X U Y AA Z H AC AE AO AG AQ AJ AR AP AH AS AF AW AT AD F D B -> Silhouette score: -0.560
  Sequence: A C E AX CA CR AZ DI CC BB DP CD BH CE DS DR DT BL CF DQ BI DJ CB BC CS BA AY F D B -> Silhouette score: -0.022
  Sequence: A C E AX CR CA AZ DW BM CG DY CI BO EA DZ CK BR CL EB DX BP CJ BN CH CS BA CB AY F D B -> Silhouette score: -0.026
  Sequ

#### Exploring PMA (Progressive Multiple Alignment)

Here, we delve into the Progressive Multiple Alignment (PMA) technique as another approach to clustering sequences, providing additional insights and comparisons.
The key idea of PMA is to iteratively build alignments by starting with the most similar pair of sequences (based here on a the edit distance) and progressively adding sequences to the alignment in order of similarity. This hierarchical approach allows us to construct a global representation of the relationships between sequences.

In [12]:
def pam(sequences, k, distance_fn, max_iter=100):
    medoids = random.sample(sequences, k)
    
    def total_cost(medoids, clusters):
        cost = 0
        for medoid, cluster in clusters.items():
            cost += sum(distance_fn(seq, medoid) for seq in cluster)
        return cost

    def assign_clusters(sequences, medoids):
        clusters = {medoid: [] for medoid in medoids}
        for seq in sequences:
            closest_medoid = min(medoids, key=lambda m: distance_fn(seq, m))
            clusters[closest_medoid].append(seq)
        return clusters

    clusters = assign_clusters(sequences, medoids)
    current_cost = total_cost(medoids, clusters)

    for iteration in range(max_iter):
        print(f"Iteration {iteration+1}")
        swap_occurred = False

        for medoid in medoids:
            for candidate in sequences:
                if candidate in medoids: 
                    continue
                
                # Tester le swap
                new_medoids = [m for m in medoids if m != medoid] + [candidate]
                new_clusters = assign_clusters(sequences, new_medoids)
                new_cost = total_cost(new_medoids, new_clusters)

                # Effectuer le swap si le coût est réduit
                if new_cost < current_cost:
                    medoids = new_medoids
                    clusters = new_clusters
                    current_cost = new_cost
                    swap_occurred = True
                    print(f"Swap performed : {medoid} -> {candidate}")
                    break 
            if swap_occurred:
                break 

        if not swap_occurred:
            print("No swap was performed, the algorithm converges.")
            break

    return clusters, medoids




In [13]:
sequences = load_sequences("log.txt", num_sequences=100)
k = 3
clusters, medoids = pam(sequences, k, edit_distance)

print("\nResults of PAM clustering  :")
for i, (medoid, cluster) in enumerate(clusters.items(), 1):
    print(f"Cluster {i} :")
    print(f"  Medoid: {medoid}")
    print(f"  Number of sequences: {len(cluster)}")
    print(f"  Sequences :")
    for seq in cluster:
        print(f"    {seq}")
    print("\n" + "-" * 50 + "\n")

Iteration 1
Swap performed : A C EC FL FX FN FS FW GA FU FQ FR FV FP FY FO FT FM GC GE GJ GI GK GH GG GF GD GL GN GO GM GW HC HE HD GX ED D B -> A C JM LN LQ LO LR LV LU LT LS LW LY LZ LX MA MD MC MB ME MM MO MN MF MR MT MU MS MV MX MW JN D B
Iteration 2
Swap performed : A C EC FL FX FS FN FZ FV FW FY FU FR FP FQ FT FO FM GC GE GJ GG GK GH GI GF GD GL GN GO GM GW GY HA HB GZ GX ED D B -> A C EC EE EG EH EF EN EP ER EQ ES EO ET EX EV EY EW EU EZ FG FJ FK FI FH FA ED D B
Iteration 3
Swap performed : A C E G I P R Q J H AC AE AO AG AQ AR AJ AP AH AF AS AW AT AD F D B -> A C E G T V W X U Y AA Z H AC AE AO AG AQ AR AJ AP AH AS AF AW AT AD F D B
Iteration 4
Swap performed : A C JM LN LQ LO LR LV LU LT LS LW LY LZ LX MA MD MC MB ME MM MO MN MF MR MT MU MS MV MX MW JN D B -> A C JM LN LQ LO LR LU LT LV LS LW LY LZ LX MA MD MC MB ME MM MQ MN MF MR MU MT MS MV MY MW JN D B
Iteration 5
Swap performed : A C EC EE EG EH EF EN EP ER EQ ES EO ET EX EV EY EW EU EZ FG FJ FK FI FH FA ED D B -> A C EC E